In [ ]:
! pip install keras-nlp datasets wandb

In [2]:
import keras_nlp
import tensorflow as tf
import datasets
from datasets import load_dataset
import numpy as np
import pandas as pd
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint
import matplotlib.pyplot as plt

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
wandb.login()

### Load and preprocess data

In [5]:
dataset = datasets.load_dataset("tatsu-lab/alpaca", split="train")
df = pd.DataFrame(dataset)
df = df[['text']]
df.head()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

,text
0,Below is an instruction that describes a task....
1,Below is an instruction that describes a task....
2,Below is an instruction that describes a task....
3,Below is an instruction that describes a task....
4,Below is an instruction that describes a task....


In [6]:
print(dataset['text'][0])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Give three tips for staying healthy.

### Response:
1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule.


In [7]:
n = int(0.9 * len(df))
train_examples = df[:n]
val_examples = df[n:]

In [8]:
train_examples.head()

,text
0,Below is an instruction that describes a task....
1,Below is an instruction that describes a task....
2,Below is an instruction that describes a task....
3,Below is an instruction that describes a task....
4,Below is an instruction that describes a task....


In [9]:
val_examples.head()

,text
46801,"Below is an instruction that describes a task,..."
46802,"Below is an instruction that describes a task,..."
46803,"Below is an instruction that describes a task,..."
46804,"Below is an instruction that describes a task,..."
46805,"Below is an instruction that describes a task,..."


In [10]:
train_examples = tf.data.Dataset.from_tensor_slices((train_examples))

val_examples = tf.data.Dataset.from_tensor_slices((val_examples))

In [11]:
BUFFER_SIZE = 20000
BATCH_SIZE = 32

In [12]:
def make_batches(ds):
    return ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [13]:
train_batches = make_batches(train_examples)
val_batches = make_batches(val_examples)

### Train model

In [14]:
num_epochs = 5

In [15]:
learning_rate = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=5e-5,
    decay_steps=train_batches.cardinality() * num_epochs,
    end_learning_rate=0.0,
)

optimizer = tf.keras.optimizers.Adam(learning_rate)

In [ ]:
wandb.init(project="gpt2-instruct-tune",
           config={
               "architecture": "gpt2",
               "dataset": "tatsu-lab/alpaca",
               "epochs": num_epochs,
               }
           )

In [17]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [19]:
preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=300
)

generator = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en",
    preprocessor=preprocessor,
)

100%|██████████| 448/448 [00:00<00:00, 288kB/s]
100%|██████████| 0.99M/0.99M [00:00<00:00, 2.64MB/s]
100%|██████████| 446k/446k [00:00<00:00, 1.51MB/s]
100%|██████████| 484/484 [00:00<00:00, 424kB/s]
100%|██████████| 475M/475M [00:14<00:00, 33.3MB/s]


In [20]:
generator.compile(
    optimizer=optimizer,
    loss=loss,
    weighted_metrics=["accuracy"],
    )

In [ ]:
history = generator.fit(train_batches,
                        validation_data=val_batches,
                        epochs=num_epochs,
                        callbacks=[WandbMetricsLogger()],
                        )

Epoch 1/5
1463/1463 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6397 - loss: 0.6284

wandb: ERROR Unable to log learning rate.


1463/1463 ━━━━━━━━━━━━━━━━━━━━ 3871s 3s/step - accuracy: 0.6397 - loss: 0.6283 - val_accuracy: 0.6734 - val_loss: 0.5499
Epoch 2/5
1463/1463 ━━━━━━━━━━━━━━━━━━━━ 3592s 2s/step - accuracy: 0.6725 - loss: 0.5414 - val_accuracy: 0.6775 - val_loss: 0.5385
Epoch 3/5
1463/1463 ━━━━━━━━━━━━━━━━━━━━ 3596s 2s/step - accuracy: 0.6821 - loss: 0.5158 - val_accuracy: 0.6795 - val_loss: 0.5342
Epoch 4/5
 188/1463 ━━━━━━━━━━━━━━━━━━━━ 50:00 2s/step - accuracy: 0.6878 - loss: 0.5004

In [ ]:
wandb.finish()

In [ ]:
metrics_df = pd.DataFrame(history.history)
metrics_df.head()

In [ ]:
metrics_df[["loss", "val_loss"]].plot()
metrics_df[["accuracy", "val_accuracy"]].plot()

In [ ]:
output = generator.generate("Formula 1 is a ", max_length=100)
print(output)

In [ ]:
prompt = "Imagine you're a detective solving a mystery in a futuristic city. Describe your first clue."

output = generator.generate(f"### Instruction:\n{prompt}\n### Response:\n", max_length=100)

print(output)

### Save model

In [ ]:
generator.save('gpt2-alpaca.keras')